### Title Generation
BIG KINDS에서 다운로드 받은 뉴스 기사 중 100000개를 선정하여 사용한다.

### 필요 라이브러리 import

In [21]:
import numpy as np
import pandas as pd
from konlpy.tag import Mecab
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.models import Model
from keras.layers import Embedding, Input, Bidirectional, LSTM, Dense, Concatenate
from keras.callbacks import ModelCheckpoint
from keras.optimizers import RMSprop
from keras.utils import pad_sequences

In [22]:
# CSV 파일 읽기
df = pd.read_csv('Merged_NewsResults.csv')[:4000]
df

,제목,본문
0,음악으로 교감하는 싱글맘과 반항아 아들 영화 '플로라 앤 썬',아일랜드 더블린의 한 동네 클럽에서 열리는 아마추어 동네 음악 경연 대회. 이제 열...
1,오뚜기와 해태 설립의 '나비효과' 김치볶음밥의 탄생[책마을],프라이팬이 한국에 전해진 것은 일제강점기였지만 대중화된 것은 1970년대부터였다. ...
2,'수산물 소비' 식당 시장 희비에도 손님은 '불안불안',지금 당장은 괜찮을 것 같아 먹긴 하지만 불안이 가시진 않는다. 나중이 걱정이죠.일...
3,"수원 국회의원, 7개 공통 공약 중 3년간 3개 이행 [경기 인천 국회의원 공약 점검 ]","사진 왼쪽부터 김승원, 백혜련, 김영진, 박광온, 김진표 국회의원. 의원실 제공 ..."
4,"수원 미완 공약, 22대 국회로 넘어가나 [경기 인천 국회의원 공약 점검 ]",수원군공항 모습. 경기일보DB 수원특례시 국회의원들이 제시한 7대 공통 공약 중 ...
...,...,...
3995,"[포토]신인선, 트로트 아닌 뮤지컬",뮤지컬 삼총사 프레스콜이 20일 오후 서울 양재동 한전아트센터에서 열렸다.프레스콜에...
3996,어벤져스 인기 다 어디갔어 벼랑끝 디즈니 동아줄은 이것,겨울왕국 어벤져스 파크 조성 디즈니플러스 등선 비용절감디즈니가 테마파크 투자액을 2...
3997,"랩그로운 다이아몬드 브랜드 ALOD, 현대백화점 목동점 입점 기념 프로모션 진행",1캐럿 이상 구매 고객에게 105만원 상당 레이어드 가드링 증정다이아몬드 전문기업 ...
3998,제주신화월드 지역상생 '드림위드 페스티벌' 눈길,제주 핫플 '안덕' 여행하고 쿠폰 받자 [파이낸셜뉴스] 국내 최대 규모 복합리...


### 간단한 데이터 전처리
한글과 알파벳을 제외한 문자들을 제거한다.

In [23]:
df['본문'] = df['본문'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z ]", "", regex=True)
df['제목'] = df['제목'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z ]", "", regex=True)

In [24]:
df

,제목,본문
0,음악으로 교감하는 싱글맘과 반항아 아들 영화 플로라 앤 썬,아일랜드 더블린의 한 동네 클럽에서 열리는 아마추어 동네 음악 경연 대회 이제 열네...
1,오뚜기와 해태 설립의 나비효과 김치볶음밥의 탄생책마을,프라이팬이 한국에 전해진 것은 일제강점기였지만 대중화된 것은 년대부터였다 오뚜기와 ...
2,수산물 소비 식당 시장 희비에도 손님은 불안불안,지금 당장은 괜찮을 것 같아 먹긴 하지만 불안이 가시진 않는다 나중이 걱정이죠일본 ...
3,수원 국회의원 개 공통 공약 중 년간 개 이행 경기 인천 국회의원 공약 점검,사진 왼쪽부터 김승원 백혜련 김영진 박광온 김진표 국회의원 의원실 제공 대 국회의...
4,수원 미완 공약 대 국회로 넘어가나 경기 인천 국회의원 공약 점검,수원군공항 모습 경기일보DB 수원특례시 국회의원들이 제시한 대 공통 공약 중 대 ...
...,...,...
3995,포토신인선 트로트 아닌 뮤지컬,뮤지컬 삼총사 프레스콜이 일 오후 서울 양재동 한전아트센터에서 열렸다프레스콜에는 연...
3996,어벤져스 인기 다 어디갔어 벼랑끝 디즈니 동아줄은 이것,겨울왕국 어벤져스 파크 조성 디즈니플러스 등선 비용절감디즈니가 테마파크 투자액을 배...
3997,랩그로운 다이아몬드 브랜드 ALOD 현대백화점 목동점 입점 기념 프로모션 진행,캐럿 이상 구매 고객에게 만원 상당 레이어드 가드링 증정다이아몬드 전문기업 KDT다...
3998,제주신화월드 지역상생 드림위드 페스티벌 눈길,제주 핫플 안덕 여행하고 쿠폰 받자 파이낸셜뉴스 국내 최대 규모 복합리조트 제...


### Dataset Split
Encoder용 데이터와 Decoder용 데이터를 분할한다.

In [25]:
# 입력 데이터와 타겟 데이터 생성
encoder_texts = df['본문'].astype(str).to_list()
decoder_texts = df['제목'].astype(str).to_list()

In [26]:
# 데이터 분리
encoder_texts_train, encoder_texts_test, decoder_texts_train, decoder_texts_test = train_test_split(
    encoder_texts, 
    decoder_texts, 
    test_size=0.2, 
    random_state=42
)

In [27]:
title_stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

content_stopwords = """아 휴 아이구 아이쿠 아이고 어 나 우리 저희 따라 의해 을 를 에 의 가 으로 로 에게 뿐이다 의거하여 근거하여 
입각하여 기준으로 예하면 예를 들면 예를 들자면 저 소인 소생 저희 지말고 하지마 하지마라 다른 물론 또한 
그리고 비길수 없다 해서는 안된다 뿐만 아니라 만이 아니다 만은 아니다 막론하고 관계없이 그치지 않다 그러나 
그런데 하지만 든간에 논하지 않다 따지지 않다 설사 비록 더라도 아니면 만 못하다 하는 편이 낫다 불문하고 향하여 
향해서 향하다 쪽으로 틈타 이용하여 타다 오르다 제외하고 이 외에 이 밖에 하여야 비로소 한다면 몰라도 외에도 이곳 
여기 부터 기점으로 따라서 할 생각이다 하려고하다 이리하여 그리하여 그렇게 함으로써 하지만 일때 할때 앞에서 중에서 
보는데서 으로써 로써 까지 해야한다 일것이다 반드시 할줄알다 할수있다 할수있어 임에 틀림없다 한다면 등 등등 제 겨우 
단지 다만 할뿐 딩동 댕그 대해서 대하여 대하면 훨씬 얼마나 얼마만큼 얼마큼 남짓 여 얼마간 약간 다소 좀 조금 다수 
몇 얼마 지만 하물며 또한 그러나 그렇지만 하지만 이외에도 대해 말하자면 뿐이다 다음에 반대로 반대로 말하자면 이와 
반대로 바꾸어서 말하면 바꾸어서 한다면 만약 그렇지않으면 까악 툭 딱 삐걱거리다 보드득 비걱거리다 꽈당 응당 해야한다 
에 가서 각 각각 여러분 각종 각자 제각기 하도록하다 와 과 그러므로 그래서 고로 한 까닭에 하기 때문에 거니와 이지만 대하여 
관하여 관한 과연 실로 아니나다를가 생각한대로 진짜로 한적이있다 하곤하였다 하 하하 허허 아하 거바 와 오 왜 어째서 무엇때문에 
어찌 하겠는가 무슨 어디 어느곳 더군다나 하물며 더욱이는 어느때 언제 야 이봐 어이 여보시오 흐흐 흥 휴 헉헉 헐떡헐떡 영차 여차 
어기여차 끙끙 아야 앗 아야 콸콸 졸졸 좍좍 뚝뚝 주룩주룩 솨 우르르 그래도 또 그리고 바꾸어말하면 바꾸어말하자면 혹은 혹시 답다 및 
그에 따르는 때가 되어 즉 지든지 설령 가령 하더라도 할지라도 일지라도 지든지 몇 거의 하마터면 인젠 이젠 된바에야 된이상 만큼 
어찌됏든 그위에 게다가 점에서 보아 비추어 보아 고려하면 하게될것이다 일것이다 비교적 좀 보다더 비하면 시키다 하게하다 할만하다 
의해서 연이서 이어서 잇따라 뒤따라 뒤이어 결국 의지하여 기대여 통하여 자마자 더욱더 불구하고 얼마든지 마음대로 주저하지 않고 곧 
즉시 바로 당장 하자마자 밖에 안된다 하면된다 그래 그렇지 요컨대 다시 말하자면 바꿔 말하면 즉 구체적으로 말하자면 시작하여 시초에 
이상 허 헉 허걱 바와같이 해도좋다 해도된다 게다가 더구나 하물며 와르르 팍 퍽 펄렁 동안 이래 하고있었다 이었다 에서 로부터 까지 예하면 
했어요 해요 함께 같이 더불어 마저 마저도 양자 모두 습니다 가까스로 하려고하다 즈음하여 다른 다른 방면으로 해봐요 습니까 했어요 말할것도 
없고 무릎쓰고 개의치않고 하는것만 못하다 하는것이 낫다 매 매번 들 모 어느것 어느 로써 갖고말하자면 어디 어느쪽 어느것 어느해 어느 년도 라 
해도 언젠가 어떤것 어느것 저기 저쪽 저것 그때 그럼 그러면 요만한걸 그래 그때 저것만큼 그저 이르기까지 할 줄 안다 할 힘이 있다 너 너희 당신 
어찌 설마 차라리 할지언정 할지라도 할망정 할지언정 구토하다 게우다 토하다 메쓰겁다 옆사람 퉤 쳇 의거하여 근거하여 의해 따라 힘입어 그 다음 
버금 두번째로 기타 첫번째로 나머지는 그중에서 견지에서 형식으로 쓰여 입장에서 위해서 단지 의해되다 하도록시키다 뿐만아니라 반대로 전후 전자 
앞의것 잠시 잠깐 하면서 그렇지만 다음에 그러한즉 그런즉 남들 아무거나 어찌하든지 같다 비슷하다 예컨대 이럴정도로 어떻게 만약 만일 위에서 
서술한바와같이 인 듯하다 하지 않는다면 만약에 무엇 무슨 어느 어떤 아래윗 조차 한데 그럼에도 불구하고 여전히 심지어 까지도 조차도 하지 않도록 
않기 위하여 때 시각 무렵 시간 동안 어때 어떠한 하여금 네 예 우선 누구 누가 알겠는가 아무도 줄은모른다 줄은 몰랏다 하는 김에 겸사겸사 하는바 
그런 까닭에 한 이유는 그러니 그러니까 때문에 그 너희 그들 너희들 타인 것 것들 너 위하여 공동으로 동시에 하기 위하여 어찌하여 무엇때문에 붕붕 
윙윙 나 우리 엉엉 휘익 윙윙 오호 아하 어쨋든 만 못하다 하기보다는 차라리 하는 편이 낫다 흐흐 놀라다 상대적으로 말하자면 마치 아니라면 쉿 그렇지 
않으면 그렇지 않다면 안 그러면 아니었다면 하든지 아니면 이라면 좋아 알았어 하는것도 그만이다 어쩔수 없다 하나 일 일반적으로 일단 한켠으로는 
오자마자 이렇게되면 이와같다면 전부 한마디 한항목 근거로 하기에 아울러 하지 않도록 않기 위해서 이르기까지 이 되다 로 인하여 까닭으로 
이유만으로 이로 인하여 그래서 이 때문에 그러므로 그런 까닭에 알 수 있다 결론을 낼 수 있다 으로 인하여 있다 어떤것 관계가 있다 관련이 있다 
연관되다 어떤것들 에 대해 이리하여 그리하여 여부 하기보다는 하느니 하면 할수록 운운 이러이러하다 하구나 하도다 다시말하면 다음으로 에 있다 
에 달려 있다 우리 우리들 오히려 하기는한데 어떻게 어떻해 어찌됏어 어때 어째서 본대로 자 이 이쪽 여기 이것 이번 이렇게말하자면 이런 이러한 
이와 같은 요만큼 요만한 것 얼마 안 되는 것 이만큼 이 정도의 이렇게 많은 것 이와 같다 이때 이렇구나 것과 같이 끼익 삐걱 따위 와 같은 사람들 
부류의 사람들 왜냐하면 중의하나 오직 오로지 에 한하다 하기만 하면 도착하다 까지 미치다 도달하다 정도에 이르다 할 지경이다 
결과에 이르다 관해서는 여러분 하고 있다 한 후 혼자 자기 자기집 자신 우에 종합한것과같이 총적으로 보면 총적으로 말하면 
총적으로 대로 하다 으로서 참 그만이다 할 따름이다 쿵 탕탕 쾅쾅 둥둥 봐 봐라 아이야 아니 와아 응 아이 참나 년 월 일 령 영 일 이 삼 사 오 육 
륙 칠 팔 구 이천육 이천칠 이천팔 이천구 하나 둘 셋 넷 다섯 여섯 일곱 여덟 아홉 령 영 이 있 하 것 들 그 되 수 이 보 않 없 나 사람 주 아니 등 
같 우리 때 년 가 한 지 대하 오 말 일 그렇 위하 때문 그것 두 말하 알 그러나 받 못하 일 그런 또 문제 더 사회 많 그리고 좋 크 따르 중 나오 가지 
씨 시키 만들 지금 생각하 그러 속 하나 집 살 모르 적 월 데 자신 안 어떤 내 내 경우 명 생각 시간 그녀 다시 이런 앞 보이 번 나 다른 어떻 여자 개 
전 들 사실 이렇 점 싶 말 정도 좀 원 잘 통하 놓""".replace("\n", "").split(" ")

In [28]:
mecab = Mecab(r"C:\mecab\mecab-ko-dic")

encoder_train_texts_tokenized = [mecab.morphs(item) for item in encoder_texts_train if item not in content_stopwords]
encoder_test_texts_tokenized = [mecab.morphs(item) for item in encoder_texts_test if item not in content_stopwords]
decoder_train_texts_tokenized = [mecab.morphs(item) for item in decoder_texts_train if item not in title_stopwords]
decoder_test_texts_tokenized = [mecab.morphs(item) for item in decoder_texts_test if item not in title_stopwords]

In [29]:
encoder_train_texts = []
encoder_test_texts = []
decoder_train_texts = []
decoder_test_texts = []

for row in encoder_train_texts_tokenized:
    for col in row:
        encoder_train_texts.append(col)
        
for row in encoder_test_texts_tokenized:
    for col in row:
        encoder_test_texts.append(col)
        
for row in decoder_train_texts_tokenized:
    for col in row:
        decoder_train_texts.append(col)
        
for row in decoder_test_texts_tokenized:
    for col in row:
        decoder_test_texts.append(col)

### 토큰화
keras에서 제공하는 Tokenizer의 text_to_sequences로 토큰화한다.

In [30]:
# 토큰화
tokenizer_encoder = Tokenizer()
tokenizer_encoder.fit_on_texts(encoder_texts_train)
num_encoder_tokens = len(tokenizer_encoder.word_index) + 1

tokenizer_decoder = Tokenizer()
tokenizer_decoder.fit_on_texts(decoder_texts_train)
num_decoder_tokens = len(tokenizer_decoder.word_index) + 1

In [31]:
encoder_sequences_train = tokenizer_encoder.texts_to_sequences(encoder_texts_train)
encoder_sequences_test = tokenizer_encoder.texts_to_sequences(encoder_texts_test)

decoder_sequences_train = tokenizer_decoder.texts_to_sequences(decoder_texts_train)
decoder_sequences_test = tokenizer_decoder.texts_to_sequences(decoder_texts_test)

### 패딩을 진행한다.

In [32]:
max_encoder_seq_length = max(len(seq) for seq in encoder_sequences_train)
max_decoder_seq_length = max(len(seq) for seq in decoder_sequences_train)

encoder_input_data_train = pad_sequences(encoder_sequences_train, maxlen=max_encoder_seq_length, padding='post')
encoder_input_data_test = pad_sequences(encoder_sequences_test, maxlen=max_encoder_seq_length, padding='post')

decoder_input_data_train = pad_sequences(decoder_sequences_train, maxlen=max_decoder_seq_length, padding='post')
decoder_input_data_test = pad_sequences(decoder_sequences_test, maxlen=max_decoder_seq_length, padding='post')

In [33]:
# decoder ont-hot encoding
decoder_output_data_train = np.zeros((len(decoder_sequences_train), max_decoder_seq_length, num_decoder_tokens), dtype='float32')
decoder_output_data_test = np.zeros((len(decoder_sequences_test), max_decoder_seq_length, num_decoder_tokens), dtype='float32')

for i, seqs in enumerate(decoder_sequences_train):
    for j, seq in enumerate(seqs):
        decoder_output_data_train[i, j, seq] = 1.

for i, seqs in enumerate(decoder_sequences_test):
    for j, seq in enumerate(seqs):
        decoder_output_data_test[i, j, seq] = 1.

In [34]:
# 모델 정의
latent_dim = 100  # 잠재 공간의 차원

# 인코더
encoder_inputs = Input(shape=(None,))
encoder_embedding = Embedding(num_encoder_tokens, latent_dim, mask_zero=True)(encoder_inputs)
encoder_bilstm = Bidirectional(LSTM(latent_dim, return_state=True))
_, forward_h, forward_c, backward_h, backward_c = encoder_bilstm(encoder_embedding)
state_h = Concatenate()([forward_h, backward_h])
state_c = Concatenate()([forward_c, backward_c])
encoder_states = [state_h, state_c]

In [35]:
# 디코더
decoder_inputs = Input(shape=(None,))
decoder_embedding = Embedding(num_decoder_tokens, latent_dim, mask_zero=True)(decoder_inputs)
decoder_lstm = LSTM(latent_dim * 2, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [36]:
# 모델 정의
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# 컴파일
model.compile(optimizer=RMSprop(), loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 훈련
checkpoint = ModelCheckpoint('seq2seq_model.h5', save_best_only=True)
model.fit(
    [encoder_input_data_train, decoder_input_data_train],
    decoder_output_data_train,
    batch_size=64,
    epochs=50,
    validation_split=0.2,
    callbacks=[checkpoint]
)

Epoch 1/50
40/40 [==============================] - 13s 119ms/step - loss: 3.3022 - accuracy: 0.0089 - val_loss: 3.2785 - val_accuracy: 0.0082
Epoch 2/50
40/40 [==============================] - 2s 59ms/step - loss: 3.1136 - accuracy: 0.0099 - val_loss: 3.2717 - val_accuracy: 0.0095
Epoch 3/50
40/40 [==============================] - 2s 54ms/step - loss: 2.9875 - accuracy: 0.0107 - val_loss: 3.2446 - val_accuracy: 0.0108
Epoch 4/50
40/40 [==============================] - 2s 54ms/step - loss: 2.8868 - accuracy: 0.0129 - val_loss: 3.2166 - val_accuracy: 0.0110
Epoch 5/50
40/40 [==============================] - 2s 51ms/step - loss: 2.7998 - accuracy: 0.0169 - val_loss: 3.2418 - val_accuracy: 0.0164
Epoch 6/50
40/40 [==============================] - 2s 55ms/step - loss: 2.7191 - accuracy: 0.0219 - val_loss: 3.2564 - val_accuracy: 0.0190
Epoch 7/50
40/40 [==============================] - 2s 52ms/step - loss: 2.6434 - accuracy: 0.0253 - val_loss: 3.2709 - val_accuracy: 0.0212
Epoch 8/50


In [39]:
from keras.models import load_model
model = load_model('seq2seq_model.h5')

# 훈련된 모델을 사용하여 시퀀스를 디코딩
encoder_model = Model(model.input[0], model.layers[6].output)

decoder_state_input_h = Input(shape=(latent_dim * 2,))
decoder_state_input_c = Input(shape=(latent_dim * 2,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_lstm = model.layers[5]
decoder_dense = model.layers[7]

decoder_outputs, state_h, state_c = decoder_lstm(
    model.input[1], initial_state=decoder_states_inputs
)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [model.input[1]] + decoder_states_inputs,
    [decoder_outputs] + decoder_states
)

TypeError: call() got an unexpected keyword argument 'initial_state'

In [ ]:
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)

    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = tokenizer_decoder.word_index['<start>']

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = tokenizer_decoder.index_word[sampled_token_index]
        decoded_sentence += sampled_char

        if sampled_char == '<end>' or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

        states_value = [h, c]

    return decoded_sentence